<a href="https://colab.research.google.com/github/sergejsrk/SparqlDatasetAnalysis/blob/main/Covid19_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPARQL queries






In [ ]:
import requests
#atlasam visas klases kopa ar tam piederoso instancu skaitu
sparql_query_all_classes="""
SELECT ?c (count(?x) as ?cx) where 
{?x a ?c}   
GROUP BY ?c 
HAVING( count(?x) >= 1 )
ORDER BY desc(?cx) 
#limit 10000
"""

r = requests.get( "https://covidontheweb.inria.fr/sparql",
                      params={'query':sparql_query_all_classes}, 
                      headers={'Accept': 'application/json'})
i=0  
for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      # onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      # self.classes.append( onto_elem )
      i+=1
print(i)

sparql_query_class_connectivity="""
SELECT ?c (count(?x) as ?cx) where 
{?x a ?c}   
GROUP BY ?c 
HAVING( count(?x) >= 1 )
ORDER BY desc(?cx) 
#limit 10000
"""

r = requests.get( "https://covidontheweb.inria.fr/sparql",
                      params={'query':sparql_query_all_classes}, 
                      headers={'Accept': 'application/json'})
i=0  
for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      # onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      # self.classes.append( onto_elem )
      i+=1
print(i)

7425
7425


In [ ]:
# Table output experiments

from IPython.display import HTML, display

def display_table(data):
    html = """<table border="1" >
              <thead>
              <tr>
                <th>URI</th>
                <th>Instance Count</th>
              </tr>              
              </thead>
           """
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td width=\"80%%\">%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

data = [["really long class name  with a full uri on top ",2],["short name",6]]
display_table(data)

URI,Instance Count
really long class name with a full uri on top,2
short name,6


# Python classes for data collection 

In [ ]:
import requests
import pickle

from IPython.display import HTML, display

# url = "https://covidontheweb.inria.fr/sparql"
# sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"

# r = requests.get(url,
#                  params={'query':sparl_query_text}, 
#                  headers={'Accept': 'application/json'})
# print(r.content)
# print(f"Response: {r.json()}")
# res_json = r.json()

# print(res_json["results"]["bindings"])

# for e in res_json["results"]["bindings"]:
#   print(e["c"]["value"] + " " + e["cx"]["value"])

class DataSet:
  def __init__(self, name, endpoint_url):
    self.name = name
    self.endpoint_url = endpoint_url
    self.classes = []
    # self.properties = []
  
  def get_classes(self):
    sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"
    r = requests.get( self.endpoint_url,
                      params={'query':sparl_query_text}, 
                      headers={'Accept': 'application/json'})
    
    for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      self.classes.append( onto_elem )

  def print_classes(self, min_amount_of_instances, class_name):
    for c in self.classes:
      if len(class_name) == 0:
        print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
               c.short_name_fragment  + "\t" + c.namespace)
      else:
        if class_name in c.url and len(class_name) > 0 : 
          print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
                c.short_name_fragment  + "\t" + c.namespace)
      if int(c.instance_count) < int(min_amount_of_instances):
        break

  def create_list_of_classes(self, min_amount_of_instances, class_name_substr):
    #if class_name_patern has len > 0 we show only classes whose name contains class_name_substr
    result = []
    for c in self.classes:
      if int(c.instance_count) < int(min_amount_of_instances):
        break

      if len(class_name_substr) == 0:
        result.append(c)
        # print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
        #        c.short_name_fragment  + "\t" + c.namespace)
      else:
        if class_name_substr in c.url and len(class_name_substr) > 0 : 
          result.append(c)
          # print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
          #       c.short_name_fragment  + "\t" + c.namespace)
      if int(c.instance_count) < int(min_amount_of_instances):
        break

    return result

  def show_used_namespaces(self):
    namespace_map = {}
    for c in self.classes:
      if c.namespace in namespace_map:
        namespace_map[c.namespace] = namespace_map[c.namespace] + 1
      else:
        namespace_map[c.namespace] = 1
    count1 = 0
    for e in namespace_map:
      print(str(e) + "\t" + str(namespace_map[e]))
      count1 = count1 + namespace_map[e]
      
    print(count1)
  
  def is_class_present_in_ds(self, class_short_name):
    for c in self.classes:
      if class_short_name in c.url:
        return True

    return False

  def get_classes_from_ds(self, class_short_name):
    res = []
    for c in self.classes:
      if class_short_name in c.url:
        res.append(c)

    return res

  def collect_connectivity(self):
    print("Collecting connectivity ...")

    s = """
    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    """
    print(s)


    i=0
    for c in self.classes:    
      print("Collecting connectivity for " + c.url + " " + str(i) + " / " + str(len(self.classes))  )
      classURI = c.url
      # sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"
      s = """        
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
          ?expr ?p ?Drug.
          ?Drug a <"""+classURI+""">.
        }
      """
      # print(s)
      r = requests.get( self.endpoint_url,
                        params={'query':s}, 
                        headers={'Accept': 'application/json'})
        
      for e in r.json()["results"]["bindings"]:
          c.connectivity = e["expr_COUNT"]["value"]
          print(e["expr_COUNT"]["value"])
      i+=1

def show_class_list_as_html_table(class_list, ds_name, ds_url):
  html = f"""<table border="1" >
            <caption> List of classes in {ds_name} ({ds_url}) </caption>
            <thead>
            <tr>
              <th>URI</th>
              <th>Short Name</th>
              <th>Instance Count</th>
              <th>Connectivity</th>
            </tr>              
            </thead>
          """
  for c in class_list:
      html += "<tr>"      
      html += "<td width=\"60%%\">%s</td>"%(c.url)
      html += "<td >%s</td>"%(c.short_name_fragment if len(c.short_name_fragment) > 0 else c.short_name )
      html += "<td >%s</td>"%(c.instance_count)
      html += "<td >%s</td>"%(c.connectivity)
      html += "</tr>"
  html += "</table>"
  display(HTML(html))


class OntologyElement:
  def __init__(self, url, instance_count):
    self.url = url
    self.instance_count = instance_count
    self.short_name = ""
    self.short_name_fragment = ""
    self.prefix = ""
    self.namespace = ""
    self.connectivity = 0
    

    if self.url.rfind("/") != -1:
      self.short_name = self.url[self.url.rfind("/") + 1:]  
      self.namespace = self.url[:self.url.rfind("/") ]  
    if self.short_name.find("#") != -1:
      self.short_name_fragment = self.short_name[self.short_name.find("#") + 1:]  
      self.namespace = self.url[:self.url.rfind("#") ]  
    
    # self.short_name = self.url[self.url.rfind("/"):]
    # self.short_name_fragment = self.short_name[self.short_name.find("#"):]


    
  

# Executing code for data collection from several endpoints


## covid19ontheweb

In [ ]:
ds1 = DataSet("covid19ontheweb", "https://covidontheweb.inria.fr/sparql")
ds1.get_classes()
ds1.collect_connectivity()
#ds1.show_used_namespaces()
# print()
#ds1.print_classes(10,"")
# show_class_list_as_html_table( ds1.create_list_of_classes(10,""), ds1.name, ds1.endpoint_url )

with open('ds1.pickle', 'wb') as f:
    pickle.dump(ds1, f)


    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    
74796550
0
9
74280960
0
1804914
1658498
658177
2099283
4
0
159513
834562
46308695
46308695
7356677
36858457
46308691
357848
357846
357846
476695
476695
0
46093
112924
874722
885158
204155
12837
21376
669270
14585
670305
0
258901
209231
463819
9885
0
337651
10960
198748
362783
196106
78400836
184031
186140
18228
8950
19532
0
37435
36017
36017
36017
369566
0
151542
150971
150971
205934
27260
27260
27260
47813
195600
251412
19701
19701
19701
19701
15965
15965
15965
153828
384893
66950
13088
13088
13088
13088
13088
13088
13088
13088
13088
13088
0
90424
61797
61797
138600
130699
137470
135888
135888
135888
135888
135888
4708
125815
93951
89741
33464
203917
285166
46696
280811
105054
224736912
231851
231851
231851
231851
231851
69055
7

In [ ]:
print()
#ds1.print_classes(10,"")
show_class_list_as_html_table( ds1.create_list_of_classes(10,""), ds1.name, ds1.endpoint_url )

URI,Short Name,Instance Count,Connectivity
http://www.w3.org/ns/oa#TextQuoteSelector,TextQuoteSelector,74796550,74796550
http://www.w3.org/ns/oa#Annotation,Annotation,74796550,0
http://www.w3.org/ns/prov#Entity,Entity,74280966,9
http://www.w3.org/ns/oa#TextPositionSelector,TextPositionSelector,74280960,74280960
http://ns.inria.fr/wasabi/ontology/Song,Song,4188779,0
http://purl.org/spar/fabio/Work,Work,1226449,1804914
http://purl.org/spar/fabio/JournalArticle,JournalArticle,1124956,1658498
http://schema.org/Observation,Observation,658177,658177
http://ns.inria.fr/wasabi/ontology/Album,Album,208743,2099283
http://www.openkg.cn/COVID-19/research/class/C1,C1,199494,4


## tweetscov19

In [ ]:
ds2 = DataSet("tweetscov19", "https://data.gesis.org/tweetscov19/sparql")
ds2.get_classes()
ds2.collect_connectivity()
# ds2.show_used_namespaces()

with open('ds2.pickle', 'wb') as f:
    pickle.dump(ds2, f)



    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    
40487590
42183854
27544245
0
20262646
13790062
590
578
322
98
4270171
2092563
1604297
1604297
1815955
1002291
621620
621620
621620
840336
514025
0
304855
304855
304855
304855
243232
243232
230427
243232
53
243232
243232
243232
1110089
241968
700021
229062
240832
378388
378388
378388
378388
697436
697436
227506
133179
1183947
1098289
238908
228682
1095879
93780
93780
93780
201332
93780
210306
138284
201332
224547
216936
224446
206282
88072
947830
947830
8
8
8
947830
506127
506127
506127
187356
288913
954484
954484
92203
946975
625093
285672
190689
90985
868074
86780
625093
187980
198506
209394
192251
184586
99832
78273
89217
184586
184586
849169
197370
175954
0
0
74396
99832
827672
0
1413165
1413165
52617
4
27570372
94561
62022
18

In [ ]:
print()
# ds2.print_classes(10,"")
show_class_list_as_html_table( ds2.create_list_of_classes(2,""), ds2.name, ds2.endpoint_url )

URI,Short Name,Instance Count,Connectivity
http://schema.org/InteractionCounter,InteractionCounter,40525292,40487590
http://rdfs.org/sioc/ns#UserAccount,UserAccount,32477877,42183854
http://www.ics.forth.gr/isl/oae/core#Entity,Entity,27570278,27544245
http://rdfs.org/sioc/ns#Post,Post,20262647,0
http://www.gsi.dit.upm.es/ontologies/onyx/ns#EmotionSet,EmotionSet,20262646,20262646
http://rdfs.org/sioc/types#Tag,Tag,13804256,13790062
http://www.w3.org/2004/02/skos/core#Concept,Concept,306,590
http://gsi.dit.upm.es/ontologies/onyx/ns#EmotionCategory,EmotionCategory,291,578
http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat,QuadMapFormat,130,322
http://www.openlinksw.com/schemas/virtrdf#array-of-QuadMapFormat,array-of-QuadMapFormat,98,98


## SIB COVID 19

In [ ]:
ds3 = DataSet("SIB COVID 19", "https://covid-19-sparql.expasy.org/sparql")
ds3.get_classes()
ds3.collect_connectivity()
# ds3.show_used_namespaces()

with open('ds3.pickle', 'wb') as f:
    pickle.dump(ds3, f)



    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    
10491650
108449
10403218
0
88598
124238
0
0
15820
90402
10980
5219322
0
0
30
9897
44007
44007
23461
27098
2856
0
3952
3384
0
10485561
30
3640
1492
8782
8782
1881
618
2444
784
1054
484
471
0
919
432
310
317
13609
13609
13609
13609
13609
8332
524
384
284
322
248
366
58151
109
343
198
198
98
125
95
182
174
2572
2572
129
8051
2572
246
451
389
128
112
63
124
119
183
118
110
162
58
95
50
295
4584
0
66
0
0
126
42
74
38
50
62
49
30
84
2039
65
53
75
47
72
72
48
106
69
69
163
18
33
51
3621
31
31
29
48
48
40
30
42
26
23
13
20
36
22
11
10
30
10
30
24
12
24
8
24
24
8
8
8
0
16
14
21
13
12
18
18
18
9
6
18
6
68
18
18
0
413
11
0
27
0
15
12
8
27
4
12
8
8
7
8
4
4
8
4
8
12
4
5
6
4
9
5
9
6
4
4
2
6
6
4
6
4
2
2
4
6
8
2
6
4
6
1
1
10
1
1
1
2
64376
2
17
3

In [ ]:
print()
# ds3.print_classes(10,"")
show_class_list_as_html_table( ds3.create_list_of_classes(10,""), ds3.name, ds3.endpoint_url )

URI,Short Name,Instance Count,Connectivity
http://biohackathon.org/resource/faldo#ExactPosition,ExactPosition,5281316,10491650
http://biohackathon.org/resource/faldo#Region,Region,5245908,108449
http://biohackathon.org/resource/faldo#Position,Position,5204334,10403218
http://biohackathon.org/resource/vg#Step,Step,5198168,0
http://biohackathon.org/resource/faldo#ForwardStrandPosition,ForwardStrandPosition,77084,88598
http://ddbj.nig.ac.jp/ontologies/nucleotide/Exon,Exon,39936,124238
http://ddbj.nig.ac.jp/ontologies/nucleotide/Mature_Peptide,Mature_Peptide,25161,0
http://ddbj.nig.ac.jp/ontologies/nucleotide/Coding_Sequence,Coding_Sequence,15820,0
http://ddbj.nig.ac.jp/ontologies/nucleotide/Protein,Protein,15820,15820
http://ddbj.nig.ac.jp/ontologies/nucleotide/Gene,Gene,14505,90402


## Covid 19 KG

In [ ]:
ds4 = DataSet("Covid 19 KG", "https://sparql.proconsortium.org/virtuoso5/sparql")
ds4.get_classes()
ds4.collect_connectivity()
# ds4.show_used_namespaces()

with open('ds4.pickle', 'wb') as f:
    pickle.dump(ds4, f)



    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    
38265866
0
2173171
36622
261782
261782
130939
98772
90292
68967
73275
19663
104755691
33426
22738
22738
19076
36622
16226
15166
247804
255373
14068
8230
10588
3555
7038
35426
5982
4886
4782
4662
564419
3959
3474
3124
1558
2842
2752
2362
2362
956
101573204
235
224
453
398
19
15762
286
322
98
172
150
1951
39663
96
60
58
46
34
32
11
20
0
18
5
8
8
8
14
12
0
4
22666
4
4
4
0
4
11
5
2
3
6
2
3
5
3
6
4
2
4
574
8
2
2
1
2
0
1
0
0
15
2
3
0
5
1
2
1
9


In [ ]:
print()
# ds4.print_classes(10,"")
show_class_list_as_html_table( ds4.create_list_of_classes(10,""), ds4.name, ds4.endpoint_url )

URI,Short Name,Instance Count,Connectivity
https://semrep.nlm.nih.gov/Entity,Entity,38265866,38265866
http://www.w3.org/2002/07/owl#Axiom,Axiom,11938498,0
https://semrep.nlm.nih.gov/Relation,Relation,2173171,2173171
http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement,Statement,394660,36622
http://biohackathon.org/resource/faldo#ExactPosition,ExactPosition,240757,261782
http://biohackathon.org/resource/faldo#Position,Position,240757,261782
http://biohackathon.org/resource/faldo#Region,Region,130891,130939
http://purl.uniprot.org/core/Domain_Extent_Annotation,Domain_Extent_Annotation,49386,98772
http://purl.uniprot.org/core/Transmembrane_Annotation,Transmembrane_Annotation,45146,90292
http://purl.uniprot.org/core/Nucleotide_Resource,Nucleotide_Resource,35426,68967


# Checking for presence of preknown interesting classes in datasets

In [ ]:

#Interesting classes from Covid on the web 
interesting_classes = ["Person",
                       "Place",
                       "Agent",
                       "Chemical", #"Chemical Substance"
                       "Disease",
                       "Procedure",
                       "Intervention",
                       "Claim",
                       "Outcome",
                       "Viruses",
                       "Biomolecule",
                       "Symptom",
                       "Chemical",
                       "Drugs",
                       "Mutation",
                       "Species",
                       "Device",
                       "Stain",
                       "Medication",
                       "Condition",
                       "Gene",
                       "Protein",
                       "Work",
                       "Position",
                       "Evidence"                    
                       ]

print("Searching for interesting classes in " + ds1.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds1.print_classes(1, c)
  print()



Searching for interesting classes in https://covidontheweb.inria.fr/sparql

Searching for Person
http://xmlns.com/foaf/0.1/Person	135614	Person		http://xmlns.com/foaf/0.1
http://ns.inria.fr/wasabi/ontology/Artist_Person	24264	Artist_Person		http://ns.inria.fr/wasabi/ontology
http://dbpedia.org/ontology/Person	5608	Person		http://dbpedia.org/ontology
http://dbpedia.org/resource/Person	203	Person		http://dbpedia.org/resource
http://dbpedia.org/ontology/MilitaryPerson	73	MilitaryPerson		http://dbpedia.org/ontology

Searching for Place
http://dbpedia.org/ontology/Place	6810	Place		http://dbpedia.org/ontology
http://dbpedia.org/ontology/PopulatedPlace	3835	PopulatedPlace		http://dbpedia.org/ontology
http://dbpedia.org/ontology/NaturalPlace	358	NaturalPlace		http://dbpedia.org/ontology
http://dbpedia.org/resource/Place	43	Place		http://dbpedia.org/resource
http://dbpedia.org/ontology/HistoricPlace	37	HistoricPlace		http://dbpedia.org/ontology

Searching for Agent
http://dbpedia.org/ontology/

In [ ]:
# interesting classes from Tweets KG
interesting_classes = ["Person", "Website", "Post", "UserAccount", "Disease", "ChemicalSubstance", "Agent", "Place" ]

print("Searching for interesting classes in " + ds2.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds2.print_classes(1, c)
  print()


Searching for interesting classes in https://data.gesis.org/tweetscov19/sparql

Searching for Person
http://dbpedia.org/class/yago/Person100007846	18	Person100007846		http://dbpedia.org/class/yago
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson	15	DUL.owl#NaturalPerson	NaturalPerson	http://www.ontologydesignpatterns.org/ont/dul/DUL.owl
http://schema.org/Person	15	Person		http://schema.org
http://xmlns.com/foaf/0.1/Person	15	Person		http://xmlns.com/foaf/0.1
http://dbpedia.org/ontology/Person	15	Person		http://dbpedia.org/ontology
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#SocialPerson	13	DUL.owl#SocialPerson	SocialPerson	http://www.ontologydesignpatterns.org/ont/dul/DUL.owl
http://dbpedia.org/ontology/PersonFunction	8	PersonFunction		http://dbpedia.org/ontology
http://umbel.org/umbel/rc/PersonWithOccupation	6	PersonWithOccupation		http://umbel.org/umbel/rc
http://dbpedia.org/class/yago/ReligiousPerson109628382	2	ReligiousPerson109628382		http://dbpedia.or

In [ ]:

# interesting classes from SIB Covid 19
interesting_classes = ["Gene", "Protein", "Work", "Position", "Evidence" ]

print("Searching for interesting classes in " + ds3.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds4.print_classes(1, c)
  print()

Searching for interesting classes in https://covid-19-sparql.expasy.org/sparql

Searching for Gene
http://purl.uniprot.org/core/Gene	7031	Gene		http://purl.uniprot.org/core

Searching for Protein
http://purl.uniprot.org/core/Protein	7041	Protein		http://purl.uniprot.org/core

Searching for Work

Searching for Position
http://biohackathon.org/resource/faldo#ExactPosition	240757	faldo#ExactPosition	ExactPosition	http://biohackathon.org/resource/faldo
http://biohackathon.org/resource/faldo#Position	240757	faldo#Position	Position	http://biohackathon.org/resource/faldo

Searching for Evidence



In [ ]:
# interesting classes from Covid 19  KG
interesting_classes = ["Stain", "Strain", "Medication", "Condition", "Gene", "Protein", "Work", "Position" ]

print("Searching for interesting classes in " + ds4.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds4.print_classes(1, c)
  print()

Searching for interesting classes in https://sparql.proconsortium.org/virtuoso5/sparql

Searching for Stain

Searching for Strain
http://purl.uniprot.org/core/Strain	33541	Strain		http://purl.uniprot.org/core

Searching for Medication

Searching for Condition

Searching for Gene
http://purl.uniprot.org/core/Gene	7031	Gene		http://purl.uniprot.org/core

Searching for Protein
http://purl.uniprot.org/core/Protein	7041	Protein		http://purl.uniprot.org/core

Searching for Work

Searching for Position
http://biohackathon.org/resource/faldo#ExactPosition	240757	faldo#ExactPosition	ExactPosition	http://biohackathon.org/resource/faldo
http://biohackathon.org/resource/faldo#Position	240757	faldo#Position	Position	http://biohackathon.org/resource/faldo



# Class presence in multiple datasets as table

In [ ]:
# Classes that are present in more than one data set

#obtain list of all classes - take short class name not full url 

all_class_names = set()

datasets = [ds1, ds2, ds3, ds4]

for ds in datasets:
  for c in ds.classes:
    if len(c.short_name_fragment) > 0:
      all_class_names.add( c.short_name_fragment ) 
    else:
      all_class_names.add( c.short_name ) 

# print(all_class_names)

# check if this name is present in each of the datasets
# name is short name
name_2_presence_in_ds = {}
for name in all_class_names:
  # print (name)
  presence = [[]] * len(datasets)
  # print(presence)
  for i in range(0,len(datasets)):
    ds = datasets[i]
    list_of_cl = ds.get_classes_from_ds(name)
    if len(list_of_cl) > 0:
      presence[i] = [(cl.short_name, cl.instance_count) for cl in list_of_cl]

  # print(name + " " +str(presence))    
  name_2_presence_in_ds[name] = presence

print(len(name_2_presence_in_ds))

print(name_2_presence_in_ds)

  


def show_class_presence_in_ds_as_html(data, datasets):
  html = f"""<table border="1" >
            <caption> Class presence in datasets </caption>
            <thead>
            <tr>
              <th>Short Class Name</th>
        """
  for ds in datasets:
    html += f"<th>{ds.name}</th>\n"
              
  html += """
            </tr>              
            </thead>
          """
  for name in data:
      html += "<tr>"      
      html += "<td >%s</td>"%(name)
      x = data[name]
      # print(x)
      i=0
      for ds in datasets:
        if len(x[i]) > 0:
          html += "<td >"
          for e in x[i]:
            html+=e[0] + " : " + e[1] + "\n<br/>" 
          html += "</td>"
        else:
          html += f"<td ></td>"
        i+=1
        
      html += "</tr>"
  html += "</table>"
  display(HTML(html))
  print(data)
  

show_class_presence_in_ds_as_html(name_2_presence_in_ds, datasets)


1933
{'WikicatAmericanFinancialLiteracyActivists': [[], [('WikicatAmericanFinancialLiteracyActivists', '1')], [], []], 'Bacteria': [[('Bacteria', '99'), ('Bacteria', '61')], [], [], []], 'Patent_Citation': [[], [], [('Patent_Citation', '1')], []], 'Q2095': [[('Q2095', '91')], [], [], []], 'QuadMapFormat': [[('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')], [('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')], [('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')], [('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')]], 'ImportantPerson110200781': [[], [('ImportantPerson110200781', '1')], [], []], 'States': [[('States', '42'), ('SupremeCourtOfTheUnitedStatesCase', '24')], [('WikicatMemberStatesOfTheUnitedNations', '3'), ('WikicatInternetCompaniesOfTheUnitedStates', '3'), ('WikicatUnitedStatesIntelligenceAgencies', '2'), ('WikicatDemocraticPartyUnitedStatesSenators', '2'), ('WikicatMediaCompan

Short Class Name,covid19ontheweb,tweetscov19,SIB COVID 19,Covid 19 KG
WikicatAmericanFinancialLiteracyActivists,,WikicatAmericanFinancialLiteracyActivists : 1,,
Bacteria,Bacteria : 99 Bacteria : 61,,,
Patent_Citation,,,Patent_Citation : 1,
Q2095,Q2095 : 91,,,
QuadMapFormat,virtrdf#QuadMapFormat : 130 virtrdf#array-of-QuadMapFormat : 98,virtrdf#QuadMapFormat : 130 virtrdf#array-of-QuadMapFormat : 98,virtrdf#QuadMapFormat : 130 virtrdf#array-of-QuadMapFormat : 98,virtrdf#QuadMapFormat : 130 virtrdf#array-of-QuadMapFormat : 98
ImportantPerson110200781,,ImportantPerson110200781 : 1,,
States,States : 42 SupremeCourtOfTheUnitedStatesCase : 24,WikicatMemberStatesOfTheUnitedNations : 3 WikicatInternetCompaniesOfTheUnitedStates : 3 WikicatUnitedStatesIntelligenceAgencies : 2 WikicatDemocraticPartyUnitedStatesSenators : 2 WikicatMediaCompaniesOfTheUnitedStates : 2 WikicatCompaniesOfTheUnitedStates : 2 WikicatSocialistStates : 2 WikicatStatesOfTheUnitedStates : 2 WikicatSingle-partyStates : 1 WikicatFederalLawEnforcementAgenciesOfTheUnitedStates : 1 WikicatAfrican-AmericanUnitedStatesSenators : 1 WikicatLGBTRightsActivistsFromTheUnitedStates : 1 WikicatComputerCompaniesOfTheUnitedStates : 1 WikicatMemberStatesOfTheCommonwealthOfNations : 1 WikicatUnitedStatesSenatorsFromDelaware : 1 WikicatVicePresidentsOfTheUnitedStates : 1 WikicatStatesAndTerritoriesEstablishedIn1837 : 1 WikicatUnitedStatesExecutiveOrders : 1 WikicatDemocraticPartyPresidentsOfTheUnitedStates : 1 WikicatPermanentRepresentativesOfTheUnitedStatesToTheUnitedNations : 1 WikicatUnitedStatesAssistantSecretariesOfState : 1 WikicatAdvertisingAgenciesOfTheUnitedStates : 1 WikicatUnitedStatesSecretariesOfHousingAndUrbanDevelopment : 1 WikicatFormerUnitedStatesExecutiveCabinetPositions : 1 WikicatUnitedStatesFootballLeagueExecutives : 1 WikicatWesternUnitedStates : 1 WikicatCommunistStates : 1 WikicatStatesAndTerritoriesEstablishedIn1949 : 1 WikicatMinorityLeadersOfTheUnitedStatesHouseOfRepresentatives : 1 WikicatSpeakersOfTheUnitedStatesHouseOfRepresentatives : 1 WikicatInformationTechnologyCompaniesOfTheUnitedStates : 1 WikicatStatesAndTerritoriesEstablishedIn660BC : 1 WikicatPresidentsOfTheUnitedStates : 1 WikicatUnitedStatesSenatorsFromIllinois : 1 WikicatAmbassadorsOfTheUnitedStates : 1 WikicatUnitedStatesAssistantSecretariesOfStateForAfricanAffairs : 1 WikicatEnglish-languageTelevisionStationsInTheUnitedStates : 1 WikicatDemocraticPartyStateGovernorsOfTheUnitedStates : 1 WikicatStateAttorneysGeneralInTheUnitedStates : 1 WikicatStateGovernorsOfTheUnitedStates : 1 WikicatMemberStatesOfTheSouthAsianAssociationForRegionalCooperation : 1 WikicatStatesAndTerritoriesEstablishedIn1947 : 1 WikicatStatesAndTerritoriesEstablishedIn1912 : 1 WikicatDemocraticPartyVicePresidentsOfTheUnitedStates : 1 WikicatRepublicanParty(UnitedStates)Politicians : 1 WikicatRepublicanPartyStateGovernorsOfTheUnitedStates : 1 WikicatMidwesternUnitedStates : 1,,
Xref,,,rdf#Xref : 10980,
Machine,Machine : 42,Machine103699975 : 4,,
Antagonist,Antagonist : 24,,,


{'WikicatAmericanFinancialLiteracyActivists': [[], [('WikicatAmericanFinancialLiteracyActivists', '1')], [], []], 'Bacteria': [[('Bacteria', '99'), ('Bacteria', '61')], [], [], []], 'Patent_Citation': [[], [], [('Patent_Citation', '1')], []], 'Q2095': [[('Q2095', '91')], [], [], []], 'QuadMapFormat': [[('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')], [('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')], [('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')], [('virtrdf#QuadMapFormat', '130'), ('virtrdf#array-of-QuadMapFormat', '98')]], 'ImportantPerson110200781': [[], [('ImportantPerson110200781', '1')], [], []], 'States': [[('States', '42'), ('SupremeCourtOfTheUnitedStatesCase', '24')], [('WikicatMemberStatesOfTheUnitedNations', '3'), ('WikicatInternetCompaniesOfTheUnitedStates', '3'), ('WikicatUnitedStatesIntelligenceAgencies', '2'), ('WikicatDemocraticPartyUnitedStatesSenators', '2'), ('WikicatMediaCompaniesOf

# Printing class names after sort by presence in datasets

In [ ]:
sorted_by_presence = []
short_name_2_presence_and_instance_count = []
for short_name in name_2_presence_in_ds:
  # print(short_name)
  #number of datasets containing clases matching this short name
  presence_count = 0 
  curr_presence = name_2_presence_in_ds[short_name]
  # print("\t" + str(curr_presence))
  inst_count_list = [0]*4
  e_ind = 0
  for e in curr_presence:
    # print(e)
    if len(e) > 0:
      presence_count += 1
      
      z = 0
      for i in e:
        z += int(i[1])
      inst_count_list[e_ind]=z
    e_ind += 1

  short_name_2_presence_and_instance_count.append(
      (
          short_name, presence_count, 
          inst_count_list[0] if len(inst_count_list)>0 else 0,
          inst_count_list[1] ,
          inst_count_list[2] ,
          inst_count_list[3] ,
       )
  )

# print(short_name_2_presence_and_instance_count)

for i in sorted(short_name_2_presence_and_instance_count, reverse = True, key=lambda x: (x[1])):
  # if i[2] > 1 and i[3]>1 and i[4]>1 and i[5]>1:
  # if (i[2] + i[3] + i[4] + i[5]) > 1000 and "Person" in i[0]:
  if (i[2] + i[3] + i[4] + i[5]) > 10 and  i[1]>=3:
    print(i)
    # print(name_2_presence_in_ds[i[0]])
    # print()



# max_score = 0
# obj_with_max_score = None
# for e in name_2_presence_in_ds:
#   print(e + " " + str(name_2_presence_in_ds[e]))
#   curr_score = 0
#   for i in range(0, len(datasets)):
#     if name_2_presence_in_ds[e][i]:
#       curr_score += 1
#   if curr_score > max_score:
#     max_score = curr_score
#     obj_with_max_score = 
  

('QuadMapFormat', 4, 228, 228, 228, 228)
('Act', 4, 554, 88, 102, 22768)
('Structure', 4, 3433, 1, 796, 17280)
('S', 4, 153645183, 20262954, 5306573, 477907)
('array-of-QuadMapFormat', 4, 98, 98, 98, 98)
('Name', 4, 821, 5, 483, 15757)
('Annotation', 4, 74796827, 5, 3382, 168168)
('Region', 4, 983, 21, 5245972, 140429)
('Property', 4, 33879, 40, 133, 235)
('Activity', 4, 229, 3, 68, 22744)
('Part', 4, 726, 22, 509, 3758)
('Statement', 4, 62609, 3, 6227, 403435)
('Value', 4, 22, 8, 8, 8)
('Ontology', 4, 39, 6, 4, 7)
('State', 4, 63094, 74, 6227, 403435)
('Domain', 4, 34, 1, 187, 51988)
('Entity', 4, 74280998, 27570358, 735, 38265867)
('Form', 4, 880, 238, 228, 228)
('Location', 4, 32, 24, 102, 6484)
('Format', 4, 318, 230, 228, 228)
('Service', 4, 79, 5, 1, 7)
('Class', 4, 18677, 19, 134, 19691)
('AnnotationProperty', 4, 277, 5, 23, 5)
('Gene', 4, 90, 1, 15021, 7031)
('Function', 4, 4948, 8, 172, 8120)
('Position', 4, 74343435, 2, 10562936, 481514)
('ObjectProperty', 4, 2338, 3, 49, 6)


# Downloading dataset files from github or preparing to use local dataset files (if present)

In [ ]:
from __future__ import print_function
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import pickle

url = 'https://github.com/sergejsrk/SparqlDatasetAnalysis/raw/main/'

last_percent_reported = None
data_root = '.' # Change me to store data elsewhere

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes=0, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  else:
    print('File found on disk. Using existing file ', filename) 
  # statinfo = os.stat(dest_filename)
  # if statinfo.st_size == expected_bytes:
  #   print('Found and verified', dest_filename)
  # else:
  #   raise Exception(
  #     'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

ds1_filename = maybe_download('ds1.pickle')
ds2_filename = maybe_download('ds2.pickle')
ds3_filename = maybe_download('ds3.pickle')
ds4_filename = maybe_download('ds4.pickle')

# test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)


file = open("ds1.pickle",'rb')
restoredDs1 = pickle.load(file)
file.close()

file = open("ds2.pickle",'rb')
restoredDs2 = pickle.load(file)
file.close()

file = open("ds3.pickle",'rb')
restoredDs3 = pickle.load(file)
file.close()

file = open("ds4.pickle",'rb')
restoredDs4 = pickle.load(file)
file.close()

# print(restoredDs1)
# print(restoredDs1.classes)
# show_class_list_as_html_table( restoredDs1.create_list_of_classes(10,""), restoredDs1.name, restoredDs1.endpoint_url )



File found on disk. Using existing file  ds1.pickle
File found on disk. Using existing file  ds2.pickle
File found on disk. Using existing file  ds3.pickle
File found on disk. Using existing file  ds4.pickle


# Presenting dataset info in a sortable and filterable way


In [ ]:
from google.colab import data_table

data_table.enable_dataframe_formatter()

import numpy as np
import pandas as pd

def create_dataframe_from_dataset(dataset_prm):
  data = []
  for c in dataset_prm.classes:
    data.append([ c.url,
               c.short_name_fragment if len(c.short_name_fragment) > 0 else c.short_name , 
               int(c.instance_count),  
               int(c.connectivity)])

  df = pd.DataFrame(
    data,
	  # index = [0,1,2],
	  columns = ['URI','Short Name', 'Instance Count', 'Connectivity']
  )

  return df


## covid19ontheweb

In [ ]:
print(f"Showing data for dataset {restoredDs1.name} ({restoredDs1.endpoint_url})")
create_dataframe_from_dataset(restoredDs1)

Showing data for dataset covid19ontheweb (https://covidontheweb.inria.fr/sparql)


,URI,Short Name,Instance Count,Connectivity
0,http://www.w3.org/ns/oa#TextQuoteSelector,TextQuoteSelector,74796550,74796550
1,http://www.w3.org/ns/oa#Annotation,Annotation,74796550,0
2,http://www.w3.org/ns/prov#Entity,Entity,74280966,9
3,http://www.w3.org/ns/oa#TextPositionSelector,TextPositionSelector,74280960,74280960
4,http://ns.inria.fr/wasabi/ontology/Song,Song,4188779,0
...,...,...,...,...
995,http://dbpedia.org/resource/Tendency,Tendency,22,3273
996,http://dbpedia.org/resource/Developer,Developer,22,655
997,http://dbpedia.org/resource/Charity,Charity,22,141
998,http://dbpedia.org/resource/Value,Value,22,4269


## tweetscov19

In [ ]:
print(f"Showing data for dataset {restoredDs2.name} ({restoredDs2.endpoint_url})")
create_dataframe_from_dataset(restoredDs2)

Showing data for dataset tweetscov19 (https://data.gesis.org/tweetscov19/sparql)


,URI,Short Name,Instance Count,Connectivity
0,http://schema.org/InteractionCounter,InteractionCounter,40525292,40487590
1,http://rdfs.org/sioc/ns#UserAccount,UserAccount,32477877,42183854
2,http://www.ics.forth.gr/isl/oae/core#Entity,Entity,27570278,27544245
3,http://rdfs.org/sioc/ns#Post,Post,20262647,0
4,http://www.gsi.dit.upm.es/ontologies/onyx/ns#E...,EmotionSet,20262646,20262646
...,...,...,...,...
915,http://dbpedia.org/class/yago/WikicatHumanRights,WikicatHumanRights,1,22855
916,http://dbpedia.org/class/yago/Writing106362953,Writing106362953,1,22855
917,http://dbpedia.org/class/yago/Sensualist109629246,Sensualist109629246,1,29066
918,http://dbpedia.org/class/yago/WikicatBisexualP...,WikicatBisexualPeople,1,29066


## SIB COVID 19

In [ ]:
print(f"Showing data for dataset {restoredDs3.name} ({restoredDs3.endpoint_url})")
create_dataframe_from_dataset(restoredDs3)

Showing data for dataset SIB COVID 19 (https://covid-19-sparql.expasy.org/sparql)


,URI,Short Name,Instance Count,Connectivity
0,http://biohackathon.org/resource/faldo#ExactPo...,ExactPosition,5281316,10491650
1,http://biohackathon.org/resource/faldo#Region,Region,5245908,108449
2,http://biohackathon.org/resource/faldo#Position,Position,5204334,10403218
3,http://biohackathon.org/resource/vg#Step,Step,5198168,0
4,http://biohackathon.org/resource/faldo#Forward...,ForwardStrandPosition,77084,88598
...,...,...,...,...
216,http://purl.uniprot.org/core/Rank,Rank,1,17
217,http://purl.uniprot.org/core/Patent_Citation,Patent_Citation,1,3
218,http://www.w3.org/ns/sparql-service-descriptio...,Dataset,1,1
219,http://purl.uniprot.org/core/Erroneous_Initiat...,Erroneous_Initiation_Annotation,1,2


## Covid 19 KG

In [ ]:
print(f"Showing data for dataset {restoredDs4.name} ({restoredDs4.endpoint_url})")
create_dataframe_from_dataset(restoredDs4)

Showing data for dataset Covid 19 KG (https://sparql.proconsortium.org/virtuoso5/sparql)


,URI,Short Name,Instance Count,Connectivity
0,https://semrep.nlm.nih.gov/Entity,Entity,38265866,38265866
1,http://www.w3.org/2002/07/owl#Axiom,Axiom,11938498,0
2,https://semrep.nlm.nih.gov/Relation,Relation,2173171,2173171
3,http://www.w3.org/1999/02/22-rdf-syntax-ns#Sta...,Statement,394660,36622
4,http://biohackathon.org/resource/faldo#ExactPo...,ExactPosition,240757,261782
...,...,...,...,...
107,http://purl.org/goodrelations/v1#ActualProduct...,ActualProductOrServicesInstance,1,5
108,http://www.ebusiness-unibw.org/ontologies/ecla...,C_AKE112003-tax,1,1
109,http://purl.uniprot.org/core/NP_Binding_Annota...,NP_Binding_Annotation,1,2
110,http://www.ebusiness-unibw.org/ontologies/ecla...,C_AKJ315005-tax,1,1
